In [1]:
import xml.etree.ElementTree as ET
import os as os
from PIL import Image
import numpy as np
import torch
import os
import cv2
import torch
import random
import numbers
import numpy as np
import pandas as pd
from PIL import Image
from torch.utils.data import Dataset, DataLoader
from torch import randperm
from torch._utils import _accumulate
import torchvision
from torchvision import datasets, transforms, models
import torchvision.transforms.functional as F
import xml.etree.ElementTree as ET
import matplotlib.pyplot as plt
import torch.nn as nn
import time
from skimage import io, transform
import sklearn.metrics as skm
from scipy.spatial import distance
import imutils
#import import_ipynb  
#from Transformation import *

In [2]:
def get_bounding_box(bound_box):
    
    xmin = int(bound_box.find('xmin').text)
    ymin = int(bound_box.find('ymin').text)
    xmax = int(bound_box.find('xmax').text)
    ymax = int(bound_box.find('ymax').text)
    return np.array((xmin,ymin,xmax,ymax,(xmin+xmax)/2,(ymin+ymax)/2,np.abs(xmax-xmin),np.abs(ymin-ymax))).reshape(-1,2)

def scaled_bounding_box(bound_box, scale):
    
    bound_box[0] = bound_box[0]/scale
    bound_box[1] = bound_box[1]/scale
    
    bound_box[2][0] = (bound_box[0, 0]+bound_box[1, 0]) /2.0
    bound_box[2][1] = (bound_box[0, 1]+bound_box[1, 1])/2.0
    
    bound_box[3][0] = torch.abs(bound_box[0, 0]-bound_box[1, 0])
    bound_box[3][1] = torch.abs(bound_box[0, 1]-bound_box[1, 1])
    return bound_box

In [3]:
def heatmap(bound_box, img_size):
    
    bound_box.float()
    
    height = bound_box[3][0]
    width  = bound_box[3][1]
    
    bound_box = scaled_bounding_box(bound_box, 4.0)
    
    image_heatmap = torch.zeros(int(img_size[0]/4),int(img_size[1]/4))
    
    k_size=8
    kernel = cv2.getGaussianKernel(k_size, 8)
    kernel = np.dot(kernel, kernel.T)
    kernel *= 100
    
    if bound_box[2][1].item()+k_size > image_heatmap.shape[0]-1:
        y_begin = image_heatmap.shape[0]-1-k_size         
    else:
        y_begin = int(bound_box[2][1].item())

    if bound_box[2][0].item()+k_size > image_heatmap.shape[1]-1:
        x_begin = image_heatmap.shape[1]-1-k_size
    else:
        x_begin = int(bound_box[2][0].item())
        
    y_end = y_begin + (k_size)
    x_end = x_begin + (k_size)
    image_heatmap[y_begin : y_end, x_begin : x_end] = torch.from_numpy(kernel)
    return image_heatmap

In [4]:
def print_dataloader(loader, num_of_image = 1, avDev = torch.device("cuda")):
    
    k=0
    for j,train_data in enumerate(loader):
        
        images = train_data['image'].to(avDev)
        heatmaps = train_data['heatmap'].to(avDev)
        plt.figure(figsize=(40,20))
        plt.subplot(1,4,1)
        plt.title('Train image' + str(k))
        plt.imshow(images[0][0].cpu())
        plt.subplot(1,4,2)
        plt.title('ball')
        plt.imshow(heatmaps[0][0].cpu().detach().numpy(), cmap= 'gray')
        plt.subplot(1,4,3)
        plt.title('goalpost')
        plt.imshow(heatmaps[0][1].cpu().detach().numpy(), cmap= 'gray')
        plt.subplot(1,4,4)
        plt.title('robot')
        plt.imshow(heatmaps[0][2].cpu().detach().numpy(), cmap= 'gray')
        plt.show()
        
        k+=1
        
        if k == num_of_image:
            return
        
    return

In [5]:
def train_model(model, mode, train_loader, criterion, optimizer, device):
    
    model.train()
    train_loss = 0.0
    
    for i_batch, sample_batched in enumerate(train_loader):
        if(mode == "blob"):
            image = sample_batched['image'].to(device)
            label = sample_batched['heatmap'].to(device)
        elif(mode == "seg"):
            image = sample_batched[0].to(device)
            label = sample_batched[1].to(device)
        
        output = model(image)
        print(output.size())
        optimizer.zero_grad()
        loss = criterion(output, label)
        train_loss += loss
        loss.backward()  
        optimizer.step()
          
    return train_loss.item(), model

In [6]:
def center_of_shape(image):
    
    centers = []
    
    kernel = np.ones((3, 3), np.uint8)
    blurred = cv2.GaussianBlur(image, (5, 5), 0)
    threshold = cv2.threshold(blurred,0.7, 255, cv2.THRESH_BINARY)[1]
    
    contours = cv2.findContours(threshold.copy().astype(np.uint8), cv2.RETR_EXTERNAL,cv2.CHAIN_APPROX_SIMPLE)
    contours = imutils.grab_contours(contours)
    
    for contour in contours:
        
        M = cv2.moments(contour)
        if(M["m00"]>0):
            cX = int(M["m10"] / M["m00"])
            cY = int(M["m01"] / M["m00"])
            centers.append((cX,cY))
            
    return torch.FloatTensor(centers)

In [7]:
def metric_calculation(actual_boxes, output_boxes, threshold_distance):
    
    num_legal_boxes = 0
    num_output_box = len(output_boxes)
    
    tp, fn, fp = 0, 0, 0
    
    for box_num in range(actual_boxes.size(0)):
        
        if(actual_boxes[box_num][2][0] > 0 or actual_boxes[box_num][2][0] >0):
            num_legal_boxes +=1
            
    flag_actual_boxes = np.zeros(num_legal_boxes)
    
    for obox_num in range(num_output_box):
        
        found = -1
        
        for abox_num in range(num_legal_boxes):
           
            a_center = actual_boxes[abox_num][2]/4.0
            o_detected = False
            o_center = output_boxes[obox_num]
            
            if(o_center[0] > 0 or o_center[1] > 0):
                o_detected = True
                
            if(o_detected):
                dist = distance.euclidean(a_center.cpu(),o_center.cpu())
                
                if(dist < threshold_distance):
                    tp += 1
                    found = abox_num
                    break
        
        if(found == -1):
            fp +=1
        
        else:
            flag_actual_boxes[found] = 1
        
    fn = np.count_nonzero(flag_actual_boxes == 0)
        
    return tp, fn, fp
    
    